# Natural Language Processing - Exercise 1 (Problem 4)

## Authors
* Charteros Eleftherios ([l.harteros@gmail.com](mailto:l.harteros@gmail.com))
* Kotitsas Sotirios ([sotiriskot9@gmail.com](mailto:sotiriskot9@gmail.com))
* Stavropoulos Petros ([pstav1993@gmail.com](mailto:pstav1993@gmail.com))
* Xenouleas Efstratios ([stratosxen@gmail.com](mailto:stratosxen@gmail.com))

In [1]:
# Import everything

import nltk
import string
import math
import copy
import random
from nltk.corpus import gutenberg, brown, stopwords
from nltk import ngrams
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from collections import Counter
from nltk.util import ngrams, pad_sequence
from pprint import pprint
from tqdm import tqdm
import numpy as np

In [2]:
# Set seed for random
random.seed(1)
np.random.seed(1)

# ALPHA is for a-smoothing
# N is the minimum frequency for a word to include it in the vocab
ALPHA = 0.01
N = 10

# Download corpuses, punctuation and stopwords
nltk.download('gutenberg')

nltk.download('punkt')
nltk.download('stopwords')

# Print the nltk version
print(nltk.__version__)
print(gutenberg.fileids())

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
3.2.5
['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


# Part A

In [3]:
# Get a specific text from gutenberg corpus
text = gutenberg.raw('austen-emma.txt')

# Or get all the gutenberg texts (for testing)
# text = ''
# for fid in gutenberg.fileids():
#   text += gutenberg.raw(fid) + ' '
# text = text[:-1]

# Remove some characters from text
text = text.translate(str.maketrans('', '', "()[]/:,;-_\"'*"))

In [4]:
# Perform sentence splitting/tokenization to the text
sentences = sent_tokenize(text)

# Perform word tokenization for each sentence and add the tokenized words to the list
sentences_tokenized = []
for sent in tqdm(sentences):
    tokens = word_tokenize(sent.lower())
    sentences_tokenized.append(tokens)

100%|██████████| 7738/7738 [00:01<00:00, 5670.35it/s]


In [5]:
# Split train - val - test
# Use 80% of the sentences as a training set

train_size = int(len(sentences_tokenized) * 0.8)
test_size = train_size + int((len(sentences_tokenized) - train_size) / 2)

train_set = sentences_tokenized[:train_size]
val_set = sentences_tokenized[train_size:test_size]
test_set = sentences_tokenized[test_size:]

In [6]:
# Flatten the sentences of the training set into words
words = [item for sublist in train_set for item in sublist]

# Find the frequency of words in the train set
freq = Counter()
freq.update(words)

# Create the vocab from the train set for words with frequency more than the specified
vocab = set()
for sent in train_set:
    vocab.update([word for word in sent if freq[word] >= N])
vocab.add('<UNK>')
vocab.add('<start>')
vocab.add('<start1>')
vocab.add('<start2>')
vocab.add('<end>')

# Create a dictionary from the set to use as a vocab
vocab = {word: i for i, word in enumerate(vocab)}
inv_vocab = {vocab[word]: word for word in vocab}

vocab_size = len(vocab)

In [7]:
# Replace unkown words in the sets
# (we could also use the defaultdict nltk module)
for i in range(len(train_set)):
    for j in range(len(train_set[i])):
        if train_set[i][j] not in vocab:
            train_set[i][j] = '<UNK>'
for i in range(len(val_set)):
    for j in range(len(val_set[i])):
        if val_set[i][j] not in vocab:
            val_set[i][j] = '<UNK>'
for i in range(len(test_set)):
    for j in range(len(test_set[i])):
        if test_set[i][j] not in vocab:
            test_set[i][j] = '<UNK>'

In [8]:
# Count ngrams
unigram_counter = Counter()
bigram_counter = Counter()
trigram_counter = Counter()

# For each sentence in the training set get the ngrams and update the according Counter
for sent in train_set:
    unigram_counter.update(sent)
    bigrams = ngrams(['<start>'] + sent + ['<end>'], 2)
    bigram_counter.update(bigrams)
    trigrams = ngrams(['<start1>'] + ['<start2>'] + sent + ['<end>'], 3)
    trigram_counter.update(trigrams)

# Add the special start tokens to the counters, in order for them to be used by the next-order ngram models
unigram_counter['<start>'] = len(train_set)
bigram_counter[('<start1>', '<start2>')] = len(train_set)

# Copy all the bigrams that start with the <start> token to bigrams that start with <start2> tokens in order
# to avoide zero probability on trigrams that start with the <start2> token
# ex. Calculating P(product| <start2>, the) = C(<start2>, the, product) / C(<start2>, the)
to_add = list()
for k in bigram_counter.keys():
    if k[0] == '<start>':
        to_add.append(('<start2>', k[1],  bigram_counter[k]))
for k in to_add:
    bigram_counter[(k[0], k[1])] = k[2]

# Part B

In [9]:
# Find the log probability of a sentence to appear on a language model using different ngram models
def predict(sent, ngram):
    if ngram not in [1, 2, 3]:
        print('Please choose either a unigram, bigram or trigram language model')
        return None

    # Find the aggregated log probability of the sentence by adding the log probabilities
    # of each ngram in the sentence
    prob = 0
    if ngram == 1:
        # Sum of all unigrams (count of all words in the train set)
        C = sum(unigram_counter.values())
        for i in range(0, len(sent) - (ngram - 1)):
            unigram_prob = (unigram_counter[sent[i]] + ALPHA) / (C + ALPHA * vocab_size)
            prob += math.log2(unigram_prob)
    elif ngram == 2:
        # Pad sentence depending on the ngram parameter
        sent = ['<start>'] + sent + ['<end>']
        for i in range(0, len(sent) - (ngram - 1)):
            bigram_prob = (bigram_counter[(sent[i], sent[i + 1])] + ALPHA) / (
                        unigram_counter[sent[i]] + ALPHA * vocab_size)
            prob += math.log2(bigram_prob)
    else:
        # Pad sentence depending on the ngram parameter
        sent = ['<start1>'] + ['<start2>'] + sent + ['<end>']
        for i in range(0, len(sent) - (ngram - 1)):
            trigram_prob = (trigram_counter[(sent[i], sent[i + 1], sent[i + 2])] + ALPHA) / (
                        bigram_counter[(sent[i], sent[i + 1])] + ALPHA * vocab_size)
            prob += math.log2(trigram_prob)

    return prob

In [10]:
# In this part we will compare the log probabilities of some of the sentences in the test dataset
# with the one's in random words from the vocab.
# We presume that the log probability of the sentences from the test dataset will be much higher
# than the random vocab words (of the same length)

# Print the probabilities of 5 sentences from the test set

print()
print('Evalutate Real-Fake Sentences Log Probabilities')
for i, sent in enumerate(test_set):
    print()
    print('Real Sentence {}'.format(i+1))
    print(' '.join(sent))
    # Create a random sentence from the vocab of the same length
    rand_sent = [inv_vocab[random.randint(0, vocab_size-1)] for _ in range(len(sent))]
    print()
    print('Fake sentence {}'.format(i+1))
    print(' '.join(rand_sent))
    # Using the unigram language model
    prob = predict(sent, 1)
    prob2 = predict(rand_sent, 1)
    print()
    print('Unigram model')
    print('Real: {} -- Fake: {}'.format(prob, prob2))
    # Using the bigram language model
    prob = predict(sent, 2)
    prob2 = predict(rand_sent, 2)
    print()
    print('Bigram model')
    print('Real: {} -- Fake: {}'.format(prob, prob2))
    # Using the trigram language model
    prob = predict(sent, 3)
    prob2 = predict(rand_sent, 3)
    print()
    print('Trigram model')
    print('Real: {} -- Fake: {}'.format(prob, prob2))
    if i == 2: break


Evalutate Real-Fake Sentences Log Probabilities

Real Sentence 1
he would <UNK> himself from <UNK> again such <UNK> <UNK> <UNK> had gone to <UNK> to be indifferent .

Fake sentence 1
look entirely looking few loved brought delighted her uncle next proof appear absence put feel observe crown completely in

Unigram model
Real: -123.32987405202242 -- Fake: -219.71561591948517

Bigram model
Real: -119.77226892195883 -- Fake: -250.720055946711

Trigram model
Real: -155.29472709170244 -- Fake: -204.26339986733186

Real Sentence 2
but he had gone to a wrong place .

Fake sentence 2
pay companion inferior means pity box odd felt therefore

Unigram model
Real: -70.83922991680878 -- Fake: -111.60699539095229

Bigram model
Real: -54.96483148523077 -- Fake: -126.25593757402588

Trigram model
Real: -68.9576437358984 -- Fake: -110.62420038197831

Real Sentence 3
there was too much <UNK> happiness in his <UNK> house woman <UNK> too amiable a form in it isabella was too much like <UNK> only in those 

# Part C

In [11]:
# Method that calculates the cross entropy of a list of sentences
# implemented according to https://en.wikipedia.org/wiki/Cross_entropy
def cross_entropy(sentences, ngram):
    # We must treat the list of sentenes as a big sentence according to the exercise so
    # loop through all the sentences and aggregate the probabilities and the sentences
    prob = 0
    size = 0
    for sent in sentences:
        prob += predict(sent, ngram)
        size += len(sent) + 1 if ngram != 1 else len(sent)

    # Divide by the size to get the cross entropy estimation
    return -(prob / size)

# Method that calculates the perplexity for a list of sentences using the language models
def perplexity(sentences, ngram):
    perplexity = 2 ** cross_entropy(sentences, ngram)
    return perplexity

In [12]:
print("2-gram model language cross-entropy : ", cross_entropy(test_set, 2))
print("2-gram model language perplexity : ", perplexity(test_set, 2))
print()
print("3-gram model language cross-entropy : ", cross_entropy(test_set, 3))
print("3-gram model language perplexity : ", perplexity(test_set, 3))

2-gram model language cross-entropy :  6.259240843606695
2-gram model language perplexity :  76.59831990811904

3-gram model language cross-entropy :  7.562849465119887
3-gram model language perplexity :  189.07954201515824


# Part D

In [17]:
# Use linear interpolation to combine the predictions of the 2 language models
def linear_predict(sent, lam=[0.5 , 0.5]):
    # Pad sentence
    sentence = ['<start1>'] + ['<start2>'] + sent + ['<end>']

    # Find the aggragated log probability of the sentence using linear interpolation of the
    # trigram and bigram probabilities for each word
    prob = 0
    for i in range(0, len(sentence) - 2):
        trigram_prob = (trigram_counter[(sentence[i], sentence[i + 1], sentence[i + 2])] + ALPHA) / (
                    bigram_counter[(sentence[i], sentence[i + 1])] + ALPHA * vocab_size)
    
        bigram_prob = (bigram_counter[(sentence[i+1], sentence[i + 2])] + ALPHA) / (
                    unigram_counter[sentence[i+1]] + ALPHA * vocab_size)

        # Linear interpolation of probabilities
        prob += math.log2(lam[0] * bigram_prob + lam[1] * trigram_prob)

        # print('P({}| {}, {})'.format(sentence[i+2],sentence[i], sentence[i+1]))
        # print('P({}| {})'.format(sentence[i+1], sentence[i]))

    return prob
    
# Cross entropy using linear interpolation of language models
def linear_cross_entropy(sentences, lam=[0.5, 0.5]):
    # We must treat the list of sentenes as a big sentence according to the exercise so
    # loop through all the sentences and aggregate the probabilities and the sentences
    prob = 0
    size = 0
    for sent in sentences:
        prob += linear_predict(sent, lam)
        size += len(sent) + 1
    # Divide by the size to get the cross entropy estimation
    return -(prob / size)

# Perplexity using linear interpolation of language models
def linear_perplexity(sentences, lam=[0.5, 0.5]):
    perplexity = 2 ** linear_cross_entropy(sentences, lam)
    return perplexity

In [18]:
print("2-gram model language cross-entropy : ", cross_entropy(test_set, 2))
print("2-gram model language perplexity : ", perplexity(test_set, 2))
print()
print("3-gram model language cross-entropy : ", cross_entropy(test_set, 3))
print("3-gram model language perplexity : ", perplexity(test_set, 3))
print()
print("linear model language cross-entropy : ", linear_cross_entropy(test_set))
print("linear model language perplexity : ", linear_perplexity(test_set))

2-gram model language cross-entropy :  6.259240843606695
2-gram model language perplexity :  76.59831990811904

3-gram model language cross-entropy :  7.562849465119887
3-gram model language perplexity :  189.07954201515824

linear model language cross-entropy :  5.839592381453616
linear model language perplexity :  57.265422501252935


In [ ]:
# Method that fine-tunes the lambda parameters for the linear interpolation for a given set of sentences
def train_linear(sentences, step):
    best = float('inf')
    best_lam = [0.5, 0.5]
    vals = np.arange(0, 1, step)
    for v in tqdm(vals):
        lamdas = [v, 1-v]
        cross = linear_cross_entropy(sentences, lam=lamdas)
        if cross < best:
            best = cross
            best_lam = lamdas
    return best_lam

In [ ]:
# Find the best lambdas
lamdas = train_linear(val_set, 0.001)

print()
print("Best lamdas : ", lamdas)

# Get the log probability on test set using the default lambdas and then the tuned lambdas
for i, sent in enumerate(test_set):
    print()
    print('Sentence {}'.format(i+1))
    print(' '.join(sent))
    print()
    print('Default lambdas log probability')
    prob = linear_predict(sent)
    print(prob)
    print("----------")
    print('Tuned lambdas log probability')
    prob = linear_predict(sent, lam=lamdas)
    print(prob)
    print("----------")
    if i == 2: break

print()
print("default linear model language cross-etropy : ", linear_cross_entropy(test_set))
print("default linear model language perplexity : ", linear_perplexity(test_set))
print()
print("tuned linear model language cross-etropy : ", linear_cross_entropy(test_set, lam=lamdas))
print("tuned linear model language perplexity : ", linear_perplexity(test_set, lam=lamdas))


100%|██████████| 1000/1000 [00:54<00:00, 17.76it/s]



Best lamdas :  [0.774, 0.22599999999999998]

Sentence 1
he would <UNK> himself from <UNK> again such <UNK> <UNK> <UNK> had gone to <UNK> to be indifferent .

Default lambdas log probability
-116.48621045502652
----------
Tuned lambdas log probability
-112.29302895024749
----------

Sentence 2
but he had gone to a wrong place .

Default lambdas log probability
-48.734729206985946
----------
Tuned lambdas log probability
-46.64366928326629
----------

Sentence 3
there was too much <UNK> happiness in his <UNK> house woman <UNK> too amiable a form in it isabella was too much like <UNK> only in those <UNK> <UNK> which always brought the other in <UNK> before him for much to have been done even had his time been <UNK> had <UNK> on however <UNK> day after <UNK> this very <UNK> <UNK> had <UNK> the history of jane <UNK> with the <UNK> which must be felt <UNK> which he did not <UNK> to feel having never believed frank churchill to be at all <UNK> emma was there so much fond <UNK> so much <UNK> 